In [415]:
import func
import svd
import pandas as pd

In [417]:
# 데이터 불러오기
def load_recipe_splited(path = 'oracle', rows = 500):
    if path == 'oracle':
        data  = func.load_recipe(n = rows)
        data2 = func.recipe_preprocessing(data)
        data3 = func.split_ingredient(data2)
        return data3

    if path == 'csv' :
        data = pd.read_csv(r'process_ingre.csv')
        return data.iloc[:rows]

recipe_raw = load_recipe_splited(path = 'csv', rows = 150000)
recipe = recipe_raw.copy()

C:\Users\kwon\AppData\Local\Temp\ipykernel_18984\511667600.py:10: DtypeWarning: Columns (68,71,74,77,80,83,86,89,90,92,93,95,96,98,99,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,180,181,182,183,184,185,186,187,188,189,190,192,193,195,196,197,198,199,200,201,202,203,204,205,207,208,210,211,213,214,216,217,218,219,220,222) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(r'process_ingre.csv')


### 단위가 "개"인 식재료 전처리 하기

### 식재료 무게 가져오기

In [418]:
# ingre_weight : 식재료별 무게 --> ingre_gram : 식재료별 무게(gram으로)
ingre_weight = pd.read_csv(r'data\ingre_weight.csv')

# 150g~200g -> 175 로 변환 후 저장
def weight2gram(ingre_weight):
    import datetime
    now = datetime.datetime.now().strftime('%Y-%m-%d_%H%M')
    def average_gram(무게):
        if "~" in 무게 :
            min, max = 무게.split("~")
            min = min.replace("g", "")
            max = max.replace("g", "")
            return (int(min)+int(max))/2
        else:
            return int(무게.replace("g", ""))
    ingre_weight["g"] = ingre_weight["무게"].apply(average_gram)
    ingre_weight.to_csv("data/ingre_weight_" + f"{now}" +".csv")
    return ingre_weight

ingre_gram = weight2gram(ingre_weight)

### 단위가 개이면 무게를 넣어주기

In [420]:
# unit_list: 단위 사전 txt -> 딕셔너리에 담기
with open("unit_gram.txt", "r", encoding="utf-8") as f:
  data = f.readlines()
df = pd.DataFrame(data, columns=["raw"])
def process(x) :
    return pd.Series(x.replace("'", "").replace(", \n", "").replace(",\n", "").replace("\'", "").replace(" ", "").split(":"))

df[["unit", "gram"]] = df['raw'].apply(process)
unit_list = dict(zip(df.unit, df.gram))

In [421]:
# change_unit 
def change_unit(row):
    def gae2gram(unit): # 식재료를 넣으면 1개의 무게가 나오는 함수 EX) gae2gram('양파') = 175
      gae_gram = dict(zip(ingre_gram.품목, ingre_gram.g ))
      return gae_gram.get(unit, 100)
    
    def unit2gram(unit): # 식재료를 넣으면 g이 나오는 함수 EX) unit2gram('t') = 5
      return unit_list.get(unit, 1)
    unit_idx = [idx for idx, col in enumerate(recipe.columns) if 'unit' in col]
    gae_idx = []
    for idx, value in enumerate(row) :
      if value == '개' and idx >= 2:
        row[idx] = gae2gram(row[idx-2])
      elif idx in unit_idx  : 
        row[idx] = unit2gram(row[idx])
      pass
    return row


In [422]:
recipe_after = recipe.apply(change_unit, axis =1) #15만개에 3분

100%|██████████| 1000/1000 [00:00<00:00, 2789.25it/s]


### 매트릭스 변환 하기

In [ ]:
from tqdm import tqdm
def recipe_food_matrix(data):
    data.index = range(len(data)) # index 초기화
    # 분수를 숫자로
    def convert_fraction_to_float(quantity):
        from fractions import Fraction
        try:
            return float(Fraction(quantity))
        except ValueError:
            return None
    def convert_unit_to_number(unit):
        unit_conversion = {
            'g': 1,
            '개': 100,
            '조금' :10
        }
        return unit_conversion.get(unit, 1)
        
    # all_ingredients: 모든 식재료 리스트
    if 'recipe_ingredients' in data.columns :
        data.drop(columns=['recipe_ingredients'])
    
    ingredient_columns = data.filter(like='ingredient')
    
    all_ingredients = set()
    for i in range(1, 20):  #20개까지만 해보기
        all_ingredients.update(data[f'ingredient{i}'].dropna().unique())

    # recipe_ingredients_df: 비어있는 레시피 X 식재료 df
    col_name = ['recipe_title'].append(list(all_ingredients))
    recipe_ingredients_df = pd.DataFrame(columns=col_name)

    # 레시피 하나씩 붙이기 
    recipe_rows = []
    for idx, row in tqdm(data.iterrows(), total = data.shape[0]) : # tqdm으로 진행상황 확인
        recipe_data = {ingredient: 0.0 for ingredient in all_ingredients}  # 모든 식재료를 None으로 초기화
        for i in range(1, 50):  
            ingredient = row[f'ingredient{i}']
            quantity = row[f'quantity{i}']
            unit = row[f'unit{i}']
            if pd.notna(ingredient) and pd.notna(quantity):
                quantity_float = convert_fraction_to_float(quantity)
                if quantity_float is not None:
                    unit_number = convert_unit_to_number(unit) if pd.notna(unit) else 1
                    recipe_data[ingredient] = quantity_float * unit_number
        recipe_rows.append(recipe_data)

    # 새로운 데이터프레임 생성 (모든 식재료를 열로 가짐)
    recipe_ingredients_df = pd.concat([pd.DataFrame([row]) for row in recipe_rows], ignore_index=True)
    recipe_ingredients_df = recipe_ingredients_df.astype('float64')
    recipe_ingredients_df['recipe_title'] = data['recipe_title']

    # RECIPE_TITLE 컬럼을 젤 앞으로
    recipe_ingredients_df = recipe_ingredients_df[['recipe_title'] + [col for col in recipe_ingredients_df.columns if col != 'recipe_title']]

    return recipe_ingredients_df

In [448]:
recipe_after

,recipe_title,ingredient1,quantity1,unit1,ingredient2,quantity2,unit2,ingredient3,quantity3,unit3,...,unit71,ingredient72,quantity72,unit72,ingredient73,quantity73,unit73,ingredient74,quantity74,unit74
0,호박 새우젓찌개 만드는 방법~★,둥근호박,1,100,청양고추,3,25.0,당근,NaN,1,...,1,NaN,NaN,1.0,NaN,NaN,1,NaN,NaN,1
1,바글바글 전복 된장찌개,된장,1,15,전복,NaN,1,양파,1/2,175.0,...,1,NaN,NaN,1.0,NaN,NaN,1,NaN,NaN,1
2,순두부찌개....오도독 씹히는 미더덕이 들어간 순두부찌개!!,순두부,NaN,1,바지락,30,1,미더덕,30,1,...,1,NaN,NaN,1.0,NaN,NaN,1,NaN,NaN,1
3,정통 바지락 순두부 찌개,순두부,1,1,바지락,100,1,양파,50,1,...,1,NaN,NaN,1.0,NaN,NaN,1,NaN,NaN,1
4,크림토마토 비프 스튜,쇠고기등심,250,1,파프리카,1,125.0,양파,1/3,175.0,...,1,NaN,NaN,1.0,NaN,NaN,1,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149995,견과류 멸치 볶음~~ 오늘은 해바라기씨 멸치 볶음,멸치,3,50,해바라기씨,2,50,포도씨유,NaN,1,...,1,NaN,NaN,1.0,NaN,NaN,1,NaN,NaN,1
149996,"만두깐풍기, 냉동만두로 저녁메뉴",만두,8,25.0,파프리카빨,NaN,1,양파,1/4,175.0,...,1,NaN,NaN,1.0,NaN,NaN,1,NaN,NaN,1
149997,두부 쉐이크 만들기,요구르트,1,100,우유,100,1,두부,1/4,1,...,1,NaN,NaN,1.0,NaN,NaN,1,NaN,NaN,1
149998,오토파지쥬스(세포해독청소부),단호박,100,1,케일,75,1,연근,75,1,...,1,NaN,NaN,1.0,NaN,NaN,1,NaN,NaN,1


In [450]:
matrix = recipe_food_matrix(recipe_after.iloc[:2000])

  0%|          | 0/2000 [00:00<?, ?it/s]

100%|██████████| 2000/2000 [00:00<00:00, 2566.75it/s]


In [488]:
matrix_raw = matrix.copy()
# nan 1000개 이상인 칼럼 제거
matrix = matrix[matrix.columns[(matrix.isna().sum() < 1000)]].copy()
# 2번 이상 쓰인 식재료만 사용
matrix = matrix[matrix.columns[((matrix != 0.0).sum(axis=0) > 1)]].copy()


In [500]:
matrix = matrix.drop(columns = 'recipe_title').copy()

In [501]:
from scipy.sparse.linalg import svds

U, S, V = svds(matrix.values)

U

In [505]:
matrix.shape

(2000, 663)

In [506]:
U

array([[-3.06619310e-05, -4.34430423e-04, -3.81647671e-04,
         6.57177264e-04, -2.71801843e-04, -2.03512601e-02],
       [-3.58508449e-06,  3.73907520e-06,  1.53217569e-06,
        -3.53639300e-07,  2.45756737e-07, -3.32737835e-07],
       [-1.97318442e-06,  3.44356910e-06,  8.84273659e-06,
        -1.02014593e-05,  9.77299241e-06, -1.84363055e-06],
       ...,
       [-1.77331577e-05,  2.07860930e-08,  3.12024654e-05,
        -8.61075485e-05,  1.01656510e-06, -1.70628103e-05],
       [-2.63667264e-02, -1.40304562e-03,  3.71847807e-03,
        -2.27269345e-05,  7.38199909e-06, -2.67898952e-06],
       [-1.17308631e-06,  2.54393654e-07,  7.01578565e-06,
        -8.27415131e-07,  3.30897500e-07, -4.47357420e-06]])